In [3]:
import os

from dotenv import load_dotenv

import snowpylot

# Import query engine functions
from snowpylot.query_engine import (
    QueryEngine,
    QueryFilter,
    SnowPilotSession,
    preview_by_date_range,
    query_by_date_range,
    query_by_username,
    query_by_organization,
    query_by_location,
    QueryFilter,
)

# Load environment variables from .env file
load_dotenv()

# Test environment setup
print("=== Environment Check ===")
user = os.environ.get('SNOWPILOT_USER')
password = os.environ.get('SNOWPILOT_PASSWORD')

if user and password:
    print('✅ Environment variables are set!')
    print(f'   SNOWPILOT_USER: {user}')
    print(f'   SNOWPILOT_PASSWORD: {"*" * len(password)}')
else:
    print('❌ Environment variables not set!')
    print('Please check your .env file contains:')
    print('SNOWPILOT_USER=your_username')
    print('SNOWPILOT_PASSWORD=your_password')

try:
    print('\n=== Testing Authentication ===')
    session = SnowPilotSession()
    if session.authenticate():
        print('✅ Authentication successful!')
    else:
        print('❌ Authentication failed - please check your credentials')

except ImportError as e:
    print(f'❌ Import error: {e}')
except Exception as e:
    print(f'❌ Error: {e}')


=== Environment Check ===
✅ Environment variables are set!
   SNOWPILOT_USER: katisthebatis
   SNOWPILOT_PASSWORD: ***********

=== Testing Authentication ===


INFO:snowpylot.query_engine:Successfully authenticated with snowpilot.org


✅ Authentication successful!


In [4]:
# Query snow pits from date range using CAAML format

# Define date range (using past dates with known data)
start_date = "2023-01-01"
end_date = "2023-01-31"
state = "MT"  # Montana - specify state as required by API

print(f"Date range: {start_date} to {end_date}")
print(f"State: {state}")
print("Format: CAAML (now the only supported format)")

# First, let's preview the query to see how many pits we'll download
print("\n=== Query Preview ===")
try:
    preview = preview_by_date_range(start_date, end_date, state=state)
    print(preview)

    # If the preview looks good, proceed with download
    if preview.estimated_count > 0:
        print(f"\n⚠️  This will download {preview.estimated_count} snow pits")
        print("Note: Large downloads may take significant time and bandwidth")

        # Download the pits (auto_approve=False will prompt for confirmation if >100 pits)
        print("\n=== Starting Download ===")
        result = query_by_date_range(start_date, end_date, state=state, auto_approve=False)

        if result.snow_pits:
            print(f"✅ Successfully downloaded {len(result.snow_pits)} snow pits!")
            print(f"Total pits found: {result.total_count}")

            # Show some basic info about the downloaded pits
            print("\n=== Download Summary ===")
            if result.download_info:
                print(f"Format: {result.download_info.get('format', 'caaml')}")
                if 'saved_file' in result.download_info:
                    print(f"Saved to: {result.download_info['saved_file']}")

            # Show info about first few pits
            print("\nFirst 5 pits:")
            for i, pit in enumerate(result.snow_pits[:5]):
                print(f"  {i+1}. Pit ID: {pit.core_info.pit_id}")
                print(f"     Date: {pit.core_info.date_time}")
                print(f"     Location: {pit.core_info.location.region if pit.core_info.location else 'Unknown'}")
                if pit.core_info.location and pit.core_info.location.elevation:
                    print(f"     Elevation: {pit.core_info.location.elevation[0]}m")
                print()

        else:
            print("❌ No pits were downloaded")
            if result.download_info.get('status') == 'cancelled':
                print("Reason: Download was cancelled by user")
    else:
        print("❌ No pits found for the specified date range")
        print("Try:")
        print("  - Different date range (e.g., winter months)")
        print("  - Different state (MT, CO, WY, etc.)")
        print("  - Check if dates are in the past (snowpilot.org doesn't have future data)")

except Exception as e:
    print(f"❌ Error during query: {e}")
    print("This might be due to:")
    print("  - Network connectivity issues")
    print("  - Authentication problems")
    print("  - Invalid date range")
    print("  - Server issues at snowpilot.org")


INFO:snowpylot.query_engine:Previewing query with filter: QueryFilter(pit_id=None, pit_name=None, date_start='2023-01-01', date_end='2023-01-31', country=None, state='MT', region=None, user_id=None, username=None, organization_id=None, organization_name=None, elevation_min=None, elevation_max=None, aspect=None, per_page=100)


Date range: 2023-01-01 to 2023-01-31
State: MT
Format: CAAML (now the only supported format)

=== Query Preview ===


INFO:snowpylot.query_engine:Previewing query: PIT_NAME=&STATE=MT&OBS_DATE_MIN=2023-01-01&OBS_DATE_MAX=2023-01-31&recent_dates=0&USERNAME=&AFFIL=&per_page=1&ADV_WHERE_QUERY=&submit=Get%20Pits
INFO:snowpylot.query_engine:Preview indicates data is available
INFO:snowpylot.query_engine:Querying CAAML pits with filter: QueryFilter(pit_id=None, pit_name=None, date_start='2023-01-01', date_end='2023-01-31', country=None, state='MT', region=None, user_id=None, username=None, organization_id=None, organization_name=None, elevation_min=None, elevation_max=None, aspect=None, per_page=100)
INFO:snowpylot.query_engine:Previewing query with filter: QueryFilter(pit_id=None, pit_name=None, date_start='2023-01-01', date_end='2023-01-31', country=None, state='MT', region=None, user_id=None, username=None, organization_id=None, organization_name=None, elevation_min=None, elevation_max=None, aspect=None, per_page=100)


Query Preview:
  Estimated pits: 10
  Date range: 2023-01-01 to 2023-01-31
  State: MT
  Username: Any
  Organization: Any
  Elevation: Any - Anym
  Format: CAAML

⚠️  This will download 10 snow pits
Note: Large downloads may take significant time and bandwidth

=== Starting Download ===


INFO:snowpylot.query_engine:Previewing query: PIT_NAME=&STATE=MT&OBS_DATE_MIN=2023-01-01&OBS_DATE_MAX=2023-01-31&recent_dates=0&USERNAME=&AFFIL=&per_page=1&ADV_WHERE_QUERY=&submit=Get%20Pits
INFO:snowpylot.query_engine:Preview indicates data is available


Preview: Will download approximately 10 pits


INFO:snowpylot.query_engine:Requesting CAAML data with query: PIT_NAME=&STATE=MT&OBS_DATE_MIN=2023-01-01&OBS_DATE_MAX=2023-01-31&recent_dates=0&USERNAME=&AFFIL=&per_page=100&ADV_WHERE_QUERY=&submit=Get%20Pits
ERROR:snowpylot.query_engine:Server error (500) - this may indicate an issue with the query parameters or server


❌ No pits were downloaded
